In [1]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import os
import skimage
import cv2

In [9]:
redPath = 'preprocessedImages/KI67/tiled/old/P07/aorta3'
nucleiPath = 'segmentations/KI67/tiled/nuclei/P7/aorta3'
savePath = "results"

In [10]:
foldersRed = [x[0] for x in os.walk(redPath)]
foldersRed = sorted(foldersRed)

foldersNuclei = [x[0] for x in os.walk(nucleiPath)]
foldersNuclei = sorted(foldersNuclei)

In [16]:
folder = foldersNuclei[0]
filesNuclei = [f for f in listdir(folder) if isfile(join(folder, f)) if "seg.npy" in f if "._" not in f]
aorta = folder.split("/")[-1]
age = folder.split("/")[-2]

In [ ]:
def intensitiesNucleus(row, nucleiMasks, redImage, redImageNorm):
    Img = np.zeros(nucleiMasks.shape, np.uint8)
    center = (int(row['centroid-1']), int(row['centroid-0']))
    radius = int((row['major_axis_length']/2) + 2)
    circle = cv2.circle(Img,center,radius,255,-1)
    nucleus = row['label']
    outside = np.where(nucleiMasks == nucleus, 0, circle)
    inside = np.where(nucleiMasks == nucleus, circle, 0)

    medianOutside = np.median(redImage[np.where(outside == 255)])
    medianOutsideNorm = np.median(redImageNorm[np.where(outside == 255)])
    median = np.median(redImage[np.where(nucleiMasks == nucleus)])
    medianNorm = np.median(redImageNorm[np.where(nucleiMasks == nucleus)])

    return (median, medianNorm, medianOutside, medianOutsideNorm)
    
def fractionNorm(row):
    if row['medianOutsideNorm'] == 0.000000:
        return row['medianNorm']/(row['medianOutsideNorm'] + 0.00012)
    else:
        return row['medianNorm']/row['medianOutsideNorm']

In [ ]:
properties = ('label', 'major_axis_length', 'minor_axis_length', 'orientation', 'centroid') 
for fileNuclei in filesNuclei:
    masks_array = np.asarray(np.load(join(folder,fileNuclei), allow_pickle = True).item()['masks'])
    redImage = skimage.io.imread(join(redPath, redFile))
    nucleiMaskRedInt = np.where(masks_array > 0, redImage, 0 )
    medianBackground = np.median(redImage[np.where(masks_array == 0)])
    redImageAdjusted = np.where(redImage <= medianBackground, medianBackground, redImage)
    redImageNorm = cv2.normalize(redImageAdjusted, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    prop_dict = regionprops_table(masks_array, properties = properties)
    df_help = pd.DataFrame(prop_dict)
    
    df_help['imageNumber'] = imageNumber
    if df_help.shape[0]:
        df_help['fractionMajorMinor'] = df_help['major_axis_length']/df_help['minor_axis_length']
        result_df = df_help.apply(lambda row: pd.Series(intensitiesNucleus(row, masks_array, redImage, redImageNorm)), axis=1)
        result_df.columns = ['median', 'medianNorm', 'medianOutside', 'medianOutsideNorm']
        df_help = pd.concat([df_help, result_df], axis=1)
        df_help['fractionNorm'] = df_help.apply(lambda row: fractionNorm(row), axis=1)
        df_help['fraction'] = (df_help['median']/df_help['medianOutside'])
        df = pd.concat([df, df_help])

In [ ]:
redImage = skimage.io.imread(join(redPath, age, aorta, redFile))
# create image, where only signal inside of nuclei segmentations is used
nucleiMaskRedInt = np.where(masks_array > 0, redImage, 0 )
# extract background signal to normalize image on noise
medianBackground = np.median(redImage[np.where(masks_array == 0)])
redImageAdjusted = np.where(redImage <= medianBackground, medianBackground, redImage)
redImageNorm = cv2.normalize(redImageAdjusted, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
prop_dict = regionprops_table(masks_array, properties = properties)
df_help = pd.DataFrame(prop_dict)

df_help['imageNumber'] = imageNumber
if df_help.shape[0]:
    df_help['fractionMajorMinor'] = df_help['major_axis_length']/df_help['minor_axis_length']
    result_df = df_help.apply(lambda row: pd.Series(intensitiesNucleus(row, masks_array, redImage, redImageNorm)), axis=1)
    result_df.columns = ['median', 'medianNorm', 'medianOutside', 'medianOutsideNorm']
    df_help = pd.concat([df_help, result_df], axis=1)
    df_help['fractionNorm'] = df_help.apply(lambda row: fractionNorm(row), axis=1)
    df_help['fraction'] = (df_help['median']/df_help['medianOutside'])
    df = pd.concat([df, df_help])

saveName = 'ki67Signal_' + age + '_' + aorta + '.csv'
df.to_csv(join(savePath, age, aorta, saveName))

In [ ]:
df = pd.DataFrame()
for folder in foldersNuclei:
    df = pd.DataFrame()
    filesNuclei = [f for f in listdir(folder) if isfile(join(folder, f)) if "_Alx568_seg.npy" in f if "._" not in f]
    aorta = folder.split("/")[-1]
    age = folder.split("/")[-2]
    original = folder.split("/")[-3]

    for fileNuclei in filesNuclei:
        nuclei = np.load(join(folder,fileNuclei), allow_pickle = True).item()

        fileRed = fileNuclei.split("_Alx568_seg.npy")[0] + "_Alx647_1.tif"
        folderRed = join(redPath)
        redImage = skimage.io.imread(join(folderRed, fileRed))
        redImageNorm = cv2.normalize(redImage, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        nucleiMasks = nuclei['masks']
        nucleiMaskRedInt = np.where(nucleiMasks > 0, redImageNorm, 0 )
        medianBackground = np.median(redImage[np.where(nucleiMasks == 0)])
        medianBackgroundNorm = np.median(redImageNorm[np.where(nucleiMasks == 0)])
        meanBackground = np.mean(redImage[np.where(nucleiMasks == 0)])
        meanBackgroundNorm = np.mean(redImageNorm[np.where(nucleiMasks == 0)])

        for i in np.unique(nucleiMasks):
            if i != 0:
                median = np.median(redImage[np.where(nucleiMasks == i)])
                medianNorm = np.median(redImageNorm[np.where(nucleiMasks == i)])
                mean = np.mean(redImage[np.where(nucleiMasks == i)])
                meanNorm = np.mean(redImageNorm[np.where(nucleiMasks == i)])

                dict = {'original': original, 'age': age, 'aorta': aorta, 'median': median, 'mean': mean, 'medianNorm': medianNorm, 'meanNorm': meanNorm, 'medianBackground': medianBackground, 'meanBackground': meanBackground, 'medianBackgroundNorm': medianBackgroundNorm, 'meanBackgroundNorm': meanBackgroundNorm,  'file': fileNuclei.split("c3_seg.npy")[0]}
                dfData = pd.DataFrame(dict, index= [0])
                df = pd.concat((df, dfData), axis=0, ignore_index=True)

    filename = "coordinatesKI67Intensity_" + original + "_" + age + "_" + aorta + "_Tiled.csv"
    df.to_csv(join("results/KI67/coordinatesKI67",filename), index = False)

In [ ]:
####Stitched

for folder in foldersNuclei:
    filesNuclei = [f for f in listdir(folder) if isfile(join(folder, f)) if "_seg.npy" in f if "._" not in f]
    properties = ('label', 'major_axis_length', 'minor_axis_length', 'orientation', 'centroid')
    df = pd.DataFrame()
    for fileNuclei in filesNuclei:
        age = folder.split("/")[-2]
        aorta = folder.split("/")[-1]
        masks_array = np.asarray(np.load(join(folder,fileNuclei), allow_pickle = True).item()['masks'])
        redFile = fileNuclei.split('_seg.npy')[0] + '.tif'
        redImage = skimage.io.imread(join(redPath, age, aorta, redFile))
        if  masks_array.shape == redImage.shape:
            nucleiMaskRedInt = np.where(masks_array > 0, redImage, 0 )
            medianBackground = np.median(redImage[np.where(masks_array == 0)])
            redImageAdjusted = np.where(redImage <= medianBackground, medianBackground, redImage)
            redImageNorm = cv2.normalize(redImageAdjusted, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
            prop_dict = regionprops_table(masks_array, properties = properties)
            df_help = pd.DataFrame(prop_dict)
            imageNumber = fileNuclei.split('_seg.npy')[0].split('-')[-1]
            df_help['imageNumber'] = imageNumber
            if df_help.shape[0]:
                df_help['fractionMajorMinor'] = df_help['major_axis_length']/df_help['minor_axis_length']
                result_df = df_help.apply(lambda row: pd.Series(intensitiesNucleus(row, masks_array, redImage, redImageNorm)), axis=1)
                result_df.columns = ['median', 'medianNorm', 'medianOutside', 'medianOutsideNorm']
                df_help = pd.concat([df_help, result_df], axis=1)
                df_help['fractionNorm'] = df_help.apply(lambda row: fractionNorm(row), axis=1)
                df_help['fraction'] = (df_help['median']/df_help['medianOutside'])
                df = pd.concat([df, df_help])
    
    if len(df) > 0:
        os.makedirs(join(savePath, age, aorta))
        saveName = 'ki67Signal_' + age + '_' + aorta + '_Stitched.csv'
        df.to_csv(join(savePath, age, aorta, saveName))